In [0]:
df = spark.read.option("header", "true").option("inferSchema", "true").csv("/Volumes/workspace/default/tutorial/ChicagoParkingTickets3.txt")
df_sample = df.limit(100000)
output_path = "/Volumes/workspace/default/tutorial/reduced_sample.csv"
df_sample.write.mode("overwrite").option("header", True).csv(output_path)


In [0]:
df = spark.read.option("header", "true").csv("/Volumes/workspace/default/tutorial/reduced_sample.csv")
df.display()

In [0]:
# storage_account_name = "jaistorageaccount"
# container_name = "sample"
# mount_point = "/mnt/rawdata"  # Choose a mount point name relevant to your data

# # Fetch service principal credentials from Databricks secrets
# application_id = dbutils.secrets.get(scope="my-secret-scope", key="application-id")
# authentication_key = dbutils.secrets.get(scope="my-secret-scope", key="application-secret")
# tenant_id = dbutils.secrets.get(scope="my-secret-scope", key="tenant-id")
# print(application_id)
# print(authentication_key)
# print(tenant_id)
# endpoint = f"https://login.microsoftonline.com/{tenant_id}/oauth2/token"

# configs = {
#   "fs.azure.account.auth.type": "OAuth",
#   "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
#   "fs.azure.account.oauth2.client.id": application_id,
#   "fs.azure.account.oauth2.client.secret": authentication_key,
#   "fs.azure.account.oauth2.client.endpoint": endpoint
# }

# # Source URI including container and storage account
# source = f"abfss://{container_name}@{storage_account_name}.dfs.core.windows.net/"

# # Mount only if not already mounted
# if not any(mount.mountPoint == mount_point for mount in dbutils.fs.mounts()):
#     dbutils.fs.mount(
#         source = source,
#         mount_point = mount_point,
#         extra_configs = configs
#     )
#     print(f"Mount successful at {mount_point}")
# else:
#     print(f"{mount_point} is already mounted")


In [0]:
# # Orchestration in Databricks
# raw_path = "/dbfs/mnt/rawdata/reduced_file.csv"
# stage1 = "/mnt/tmp/stage1"
# stage2 = "/mnt/tmp/stage2"
# stage3 = "/mnt/tmp/stage3"
# stage4 = "/mnt/tmp/stage4"
# train_path = "/mnt/tmp/train"
# test_path = "/mnt/tmp/test"
# model_path = "/mnt/models"
# registered_name = "my_spark_native"

# dbutils.notebook.run("01_load_data", 0, {"raw_path": raw_path, "output_path": stage1})
# dbutils.notebook.run("02_feature_engineering", 0, {"input_path": stage1, "output_path": stage2})
# dbutils.notebook.run("03_replace_missing", 0, {"input_path": stage2, "output_path": stage3})
# dbutils.notebook.run("04_select_features", 0, {"input_path": stage3, "output_path": stage4})
# dbutils.notebook.run("05_split_data", 0, {"input_path": stage4, "train_path": train_path, "test_path": test_path})
# dbutils.notebook.run("06_train_and_register", 0, {
#     "train_path": train_path, 
#     "test_path": test_path,
#     "model_output_path": model_path,
#     "registered_model_name": registered_name
# })


In [0]:
# import mlflow.pyfunc
# import pandas as pd

# # Load model by name and version (use latest version by default)
# model_name = "my_spark_native_model"
# model_version = 1  # or omit to get latest version

# model_uri = f"models:/{model_name}/{model_version}"  

# # Load model as a PyFunc model (generic for most flavors)
# model = mlflow.pyfunc.load_model(model_uri)

# # Predict on new data - input as Pandas DataFrame or Spark DataFrame (converted to pandas)
# data = pd.DataFrame({
#     "Issued_year": [2023],
#     "Issued_month": [5],
#     "Issued_day": [3],
#     "Issued_hour": [10],
#     "Community_Name": [1],
#     "Sector": [1],
#     "Side": [1],
#     "Hardship_Index": [50.5],
#     "Per_capita_income": [15957.00],
#     "Percent_unemployed": [5.0],
#     "Percent_without_diploma": [10.0],
#     "Percent_households_below_poverty": [12.0],
#     "Neighborhood": [1],
#     "Ward": [1],
#     "ZIP": [12345],
#     "Police_District": [1],
#     "Plate_Type": [1],
#     # Add other features your model requires
# })

# # If Spark DataFrame, convert to pandas
# if hasattr(data, "toPandas"):
#     data = data.toPandas()

# predictions = model.predict(data)
# print(predictions)

